***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/travel_insurance.csv')
df = df.dropna()
df = df.drop(columns=['Gender'])

encoder = OrdinalEncoder()
df['Agency'] = encoder.fit_transform(df[['Agency']])
df['Agency Type'] = encoder.fit_transform(df[['Agency Type']])
df['Distribution Channel'] = encoder.fit_transform(df[['Distribution Channel']])
df['Product Name'] = encoder.fit_transform(df[['Product Name']])
df['Destination'] = encoder.fit_transform(df[['Destination']])

df['Claim'] = encoder.fit_transform(df[['Claim']])
X = df.loc[:, df.columns != 'Claim']
y = df['Claim']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)

0.0    17580
1.0    17580
Name: Claim, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)
grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')

grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

Best Parameters: {'C': 100}
Best Score: 0.5757370885647184
Best Estimator: LinearSVC(C=100, random_state=42)


In [ ]:
best = LinearSVC(C=100, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.66061565 0.70093456 0.51361763 0.49751858 0.33396505 0.37092101
 0.58375099 0.48820981 0.3327007  0.38201661]
Mean CV Accuracy: 0.4864250611601443
Training Set Accuracy: 0.6986348122866894


##**Extracting weights and intercept values**

In [ ]:
for (intercept, coef) in zip(best.intercept_, best.coef_):
    s = "y = {0:.3f}".format(intercept)
    for (i, c) in enumerate(coef):
      s += " + {0} * x{1}".format(c, i)

    print(s)

y = -0.367 + -0.11928742631613651 * x0 + 0.7638556922872276 * x1 + 0.45953757346923935 * x2 + 0.0058205499799236764 * x3 + -0.0008785689070541346 * x4 + 0.0023608547819608337 * x5 + 0.009750449284590068 * x6 + -0.006027012882020856 * x7 + -0.00820472580465281 * x8


In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['Claim'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Claim_Predicted': predicted_label})
index_class = results_df.index[results_df['Claim_Predicted'] == 0]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

12454


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.2 MB/s eta 0:00:00
  Created wheel for z3: filename=z3-0.2.0-py3-none-any.whl size=26629 sha256=3716f2c7efaada0e38794818c7d0685858d00fad5f662ec33bc7f35cc69e7a05
  Stored in directory: /root/.cache/pip/wheels/5a/b2/60/55b07a5084cad7ab411e395fb1440a2b1a19598bff535a3955
Successfully built z3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.8 MB/s eta 0:00:00


##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    original_value1 = og_x_np[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    original_value2 = og_x_np[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    original_value3 = og_x_np[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    opt.add(And(X[4] >= 30, X[4] <= 100))

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    opt.add(And(X[6] >= 20, X[6] <= 80))

    opt.add(And(X[7] >= 30, X[7] <= 60))

    original_value8 = og_x_np[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)
    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    slack_duration = Real('slack_duration')
    slack_duration_constr = (slack_duration >=0)
    opt.add(And(X[4] > 0, X[4] >= 30 - slack_duration, X[4] <= 100 + slack_duration))
    opt.add(slack_duration_constr)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    slack_net = Real('slack_net')
    slack_net_constr = (slack_net >=0)
    opt.add(And(X[6] > 0, X[6] < 187, X[6] >= 20 - slack_net, X[6] <= 80 + slack_net))
    opt.add(slack_net_constr)

    slack_commision = Real('slack_commision')
    slack_commision_constr = (slack_commision >=0)
    opt.add(And(X[7] > 0, X[7] >= 30 - slack_commision, X[7] <= 60 + slack_commision))
    opt.add(slack_commision_constr)

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    objective =  (slack_duration + slack_net + slack_commision)
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized distance: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {cost_optimized}")

        slack_duration = model[slack_duration].as_decimal(15)
        print(f"slack_duration value: {slack_duration}")

        slack_net = model[slack_net].as_decimal(15)
        print(f"slack_net value: {slack_net}")

        slack_commision = model[slack_commision].as_decimal(15)
        print(f"slack_commision value: {slack_commision}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_duration, slack_net, slack_commision

    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()
    original_f, x_prime_optimized, cost, cost_optimized, slack_duration, slack_net, slack_commision = step2(W, B, row)
    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_duration] + [slack_net] + [slack_commision]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_duration', 'Slack_net', 'Slack_commision']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2, slack_value3):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    original_value1 = row[1]
    if original_value1 is not None:
        opt.add(X[1] == original_value1)

    original_value2 = row[2]
    if original_value2 is not None:
        opt.add(X[2] == original_value2)

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    opt.add(And(X[4] >= 30 - slack_value1, X[4] <= 100 + slack_value1))

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    opt.add(And(X[6] >= 20 - slack_value2, X[6] <= 80 + slack_value2, X[6] < 187))

    opt.add(And(X[7] >= 30 - slack_value3, X[7] <= 60 + slack_value3))

    original_value8 = row[8]
    if original_value8 is not None:
        opt.add(X[8] == original_value8)

    dot_product = Sum([W[i] * X[i] for i in range(n)])
    inequality = dot_product + B > 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective)
        print(f"Distance: {l1_distance_1}")

        cost_optimized = model.eval(objective)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")

        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_duration'].iloc[i]
    slack_value2 = last['Slack_net'].iloc[i]
    slack_value3 = last['Slack_commision'].iloc[i]

    start_time = time.time()
    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2, slack_value3)
    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

Streaming output truncated to the last 5000 lines.
Distance: 2
Objective: 2
Optimized x': ['5', '1', '0', '9', '88', '43', '186.0130447392079', '30', '70']
Distance: 2
Objective: 2
Optimized x': ['7', '0', '1', '21', '38', '30', '186.90194191485756', '30', '118']
Distance: 2
Objective: 2
Optimized x': ['7', '0', '1', '21', '62', '30', '186.90194191485756', '30', '118']
Distance: 2
Objective: 2
Optimized x': ['7', '0', '1', '21', '96', '30', '186.90194191485756', '30', '118']
Distance: 2
Objective: 2
Optimized x': ['7', '0', '1', '21', '30', '30', '186.90194191485756', '30', '118']
Distance: 3
Objective: 3
Optimized x': ['7', '0', '1', '21', '45', '30', '186.90194191485756', '30', '118']
Distance: 2
Objective: 2
Optimized x': ['7', '0', '1', '21', '64', '30', '186.90194191485756', '30', '118']
Distance: 2
Objective: 2
Optimized x': ['11', '0', '1', '21', '30', '11', '186.9232518058838', '30', '57']
Distance: 3
Objective: 3
Optimized x': ['11', '0', '1', '21', '30', '11', '186.7201143423

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 & l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

L0 0.014871719469159224


In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step1_optimized.csv')

d = pd.concat([d1, d3_initial])
l1_runtime = d.loc[:, 'Runtime'].mean()
print('L1 runtime', l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

66.71752047534929


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Travel/New-Step2.csv')
number_of_slacks_per_record = 3

count_s1 = 0
count_s2 = 0
count_s3 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_duration'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_duration'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_net'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_net'].iloc[i] == 0:
        count_s2 = 0
    if relaxed_df['Slack_commision'].iloc[i] != 0:
        count_s3 = 1
    elif relaxed_df['Slack_commision'].iloc[i] == 0:
        count_s3 = 0

    total_count = count_s1 + count_s2 + count_s3
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)

33.50583704417071
